In [1]:
import pandas as pd
import numpy as np

In [9]:
# autoregression = pd.read_csv("../outputs/predictions/sequifier-default-best-1575-all-autoregression-predictions.csv")
onestep = pd.read_csv("../outputs/predictions/sequifier-default4-best-50-predictions.csv")
onestep.shape

(658, 9)

In [10]:
onestep

,sequenceId,Australia,British,Canada,Switzerland,China,Japan,NewZealand,Singapore
0,1,0.745555,1.418498,0.775707,0.981409,0.152654,0.008043,0.744219,0.723266
1,1,0.746054,1.416595,0.775079,0.980459,0.152597,0.008025,0.746951,0.722826
2,1,0.745950,1.416502,0.774998,0.980283,0.152592,0.008024,0.746813,0.722733
3,1,0.744270,1.422976,0.773791,0.973536,0.151977,0.007987,0.746639,0.722173
4,1,0.739404,1.421342,0.777702,0.971692,0.152791,0.008042,0.734201,0.720618
...,...,...,...,...,...,...,...,...,...
653,1,0.768555,1.320229,0.796245,1.058071,0.151191,0.009284,0.751899,0.724056
654,1,0.768690,1.320383,0.795257,1.058782,0.151224,0.009273,0.751014,0.723921
655,1,0.768903,1.319820,0.795534,1.056867,0.151210,0.009271,0.753133,0.723120
656,1,0.770803,1.322302,0.798628,1.062040,0.151116,0.009357,0.755829,0.724380


In [3]:
target = pd.read_csv("../data/exchange_rate.txt").drop(columns=["itemPosition"])

In [19]:
target2 = pd.read_parquet("../data/exchange_rate-split2.parquet")["target"].values.reshape(onestep.shape[0], 8)
import json
with open("../configs/ddconfigs/exchange_rate.json", "r") as f:
    min_max_values = json.loads(f.read())["min_max_values"]

In [ ]:
def invert_normalization(min_max_values, values, target_column):
    min_ = min_max_values[target_column]["min"]
    max_ = min_max_values[target_column]["max"]
    return np.array(
        [(((v + 0.8) / 1.6) * (max_ - min_)) + min_ for v in values.flatten()]
    ).reshape(*values.shape)

In [20]:
min_max_values

{'Australia': {'min': 0.483297, 'max': 1.102536},
 'British': {'min': 1.211534, 'max': 2.109},
 'Canada': {'min': 0.618582, 'max': 1.091524},
 'Switzerland': {'min': 0.548617, 'max': 1.374079},
 'China': {'min': 0.109292, 'max': 0.237954},
 'Japan': {'min': 0.006254, 'max': 0.013202},
 'NewZealand': {'min': 0.393153, 'max': 0.882379},
 'Singapore': {'min': 0.523834, 'max': 0.832556}}

In [18]:
target2["target"].values.reshape(onestep.shape[0], 8)

array([[-0.07493972, -0.33181959, -0.24609529, ..., -0.34312032,
         0.31746146,  0.20657031],
       [-0.07493972, -0.33181959, -0.24609529, ..., -0.34312032,
         0.31746146,  0.20657031],
       [-0.06934576, -0.31717213, -0.24157212, ..., -0.34173863,
         0.33417684,  0.21554149],
       ...,
       [-0.18365122, -0.77874014, -0.3853259 , ..., -0.26090961,
         0.1894748 ,  0.06267386],
       [-0.1801424 , -0.75974978, -0.37169547, ..., -0.26850892,
         0.19026626,  0.06853545],
       [-0.18627121, -0.76011704, -0.37525785, ..., -0.2701209 ,
         0.17962414,  0.06606332]])

In [4]:
columns = ["Australia", "British", "Canada", "Switzerland", "China", "Japan", "NewZealand", "Singapore"]

In [5]:
#np.mean(np.abs(autoregression[columns].values - target[columns].tail(738).values))

In [6]:
np.mean(np.abs( (np.tile(target[columns].tail(659).head(1).values, 738)).reshape(738, len(columns)) - target[columns].tail(738).values))

0.03041429962737127

In [7]:
np.mean(np.abs(onestep[columns].values - target[columns].tail(658).values))

0.017093471173100585

In [8]:
np.mean(np.abs(target[columns].tail(659).head(658).values - target[columns].tail(658).values))

0.002476956496960486